In [3]:
from prophet import Prophet
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import joblib

In [4]:
df = pd.read_csv('http_metrics.csv' )

In [5]:
df.head()

,ds,name,instances,display_host,record_count,mult_records,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,web_response_time,throughput
0,2024-04-15 23:32:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,30.099163,0.000091,0.785708,2.049792,0.007541,3898.0
1,2024-04-15 23:33:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,32.763374,0.000103,3.822411,8.938635,0.008417,3917.5
2,2024-04-15 23:34:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,34.776276,0.000110,3.822411,16.068309,0.009546,3993.0
3,2024-04-15 23:35:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,33.072502,0.000110,3.659296,8.886156,0.009014,3991.5
4,2024-04-15 23:36:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,42.575750,0.000107,15.111876,135.348995,0.012279,3915.5


In [6]:
df.shape

(43272, 12)

In [7]:
ts=df[['ds','web_response_time']]

In [8]:
ts['ds'] = pd.to_datetime(ts['ds'])

<ipython-input-8-adb4a517f4fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['ds'] = pd.to_datetime(ts['ds'])


In [9]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43272 entries, 0 to 43271
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ds                 43272 non-null  datetime64[ns]
 1   web_response_time  43272 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 676.2 KB


In [10]:
Prophet

prophet.forecaster.Prophet

In [11]:
ts.rename(columns={"web_response_time": "y"}, inplace=True)


<ipython-input-11-b65ad554aa8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts.rename(columns={"web_response_time": "y"}, inplace=True)


In [12]:

model = Prophet(interval_width=0.99, weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=True)
model.fit(ts)
forecast = model.predict(ts)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpai9odcus/syvc216r.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpai9odcus/i_ti9dwl.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20267', 'data', 'file=/tmp/tmpai9odcus/syvc216r.json', 'init=/tmp/tmpai9odcus/i_ti9dwl.json', 'output', 'file=/tmp/tmpai9odcus/prophet_model_19ojigk/prophet_model-20240525231953.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:19:53 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:20:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [13]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-04-15 23:32:00,24.887745,-103.182833,103.842488,24.887745,24.887745,-21.117039,-21.117039,-21.117039,-3.256820,-3.256820,-3.256820,-17.860218,-17.860218,-17.860218,0.0,0.0,0.0,3.770707
1,2024-04-15 23:33:00,24.881220,-103.390843,102.433628,24.881220,24.881220,-21.159798,-21.159798,-21.159798,-3.297615,-3.297615,-3.297615,-17.862183,-17.862183,-17.862183,0.0,0.0,0.0,3.721422
2,2024-04-15 23:34:00,24.874695,-106.173130,100.575815,24.874695,24.874695,-21.202568,-21.202568,-21.202568,-3.338423,-3.338423,-3.338423,-17.864145,-17.864145,-17.864145,0.0,0.0,0.0,3.672127
3,2024-04-15 23:35:00,24.868170,-98.810669,105.293922,24.868170,24.868170,-21.245344,-21.245344,-21.245344,-3.379238,-3.379238,-3.379238,-17.866105,-17.866105,-17.866105,0.0,0.0,0.0,3.622826
4,2024-04-15 23:36:00,24.861645,-113.031044,109.142999,24.861645,24.861645,-21.288118,-21.288118,-21.288118,-3.420054,-3.420054,-3.420054,-17.868064,-17.868064,-17.868064,0.0,0.0,0.0,3.573527


In [14]:
# Предикт делаем на полную исходную выборку:
performance = pd.merge(ts, forecast[['ds','yhat_lower', 'yhat',  'yhat_upper']], on='ds')
performance.head()


,ds,y,yhat_lower,yhat,yhat_upper
0,2024-04-15 23:32:00,0.007541,-103.182833,3.770707,103.842488
1,2024-04-15 23:33:00,0.008417,-103.390843,3.721422,102.433628
2,2024-04-15 23:34:00,0.009546,-106.173130,3.672127,100.575815
3,2024-04-15 23:35:00,0.009014,-98.810669,3.622826,105.293922
4,2024-04-15 23:36:00,0.012279,-113.031044,3.573527,109.142999


In [15]:
performance.rename(columns={'y': 'y_real', 'yhat_lower': 'y_pred_lower', 'yhat': 'y_pred', 'yhat_upper': 'y_pred_upper'}, inplace=True)
performance = performance[['ds', 'y_pred_lower', 'y_pred', 'y_pred_upper', 'y_real']]
performance.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real
0,2024-04-15 23:32:00,-103.182833,3.770707,103.842488,0.007541
1,2024-04-15 23:33:00,-103.390843,3.721422,102.433628,0.008417
2,2024-04-15 23:34:00,-106.173130,3.672127,100.575815,0.009546
3,2024-04-15 23:35:00,-98.810669,3.622826,105.293922,0.009014
4,2024-04-15 23:36:00,-113.031044,3.573527,109.142999,0.012279


In [16]:
# Вычисляем стандартное отклонение предсказанных значений
std_dev = performance['y_pred'].std()

# Определяем аномалии с более строгим порогом
performance['anomaly'] = performance.apply(
    lambda row: 1 if (row['y_real'] < (row['y_pred_lower'] - 20 * std_dev)) or (row['y_real'] > (row['y_pred_upper'] + 20 * std_dev)) else 0, axis=1)

anomalies = performance[performance['anomaly'] == 1].sort_values(by='ds')
anomalies.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real,anomaly
7755,2024-04-21 08:59:00,-70.186027,47.955422,165.433273,1923.77050,1
9295,2024-04-22 10:39:00,-63.021982,54.742215,170.673232,645.01074,1
9640,2024-04-22 16:24:00,-48.550112,58.685347,170.219133,656.48834,1
9742,2024-04-22 18:06:00,-55.407283,49.709833,150.102939,1124.04420,1
9743,2024-04-22 18:07:00,-53.432078,49.604923,159.425850,952.43780,1


In [17]:
performance['color'] = np.where(performance['anomaly']== 1, 'red', 'green')
performance['name'] = np.where(performance['anomaly'] == 1, 'Anomaly','Within Confidence Interval')

In [18]:
# Преобразуем исходные данные в DataFrame для Plotly
performance['timestamp'] = performance['ds']
performance['target'] = performance['y_real']
ts_df = performance[['timestamp', 'target']]

# Создаем график временного ряда
fig = px.line(ts_df, x='timestamp', y='target', title='Time Series with Anomalies')

# Добавляем аномалии на график
anomaly_points = performance[performance['anomaly'] == 1]
fig.add_scatter(x=anomaly_points['timestamp'], y=anomaly_points['target'],
                mode='markers', marker=dict(color='red', size=10), name='Anomalies')


In [23]:
# Подготовка итогового DataFrame
result = performance[['ds', 'y_real', 'anomaly']]
result.rename(columns={'ds': 'timestamp', 'y_real': 'Web Response'}, inplace=True)
result['model'] = 'prophet'

result.rename(columns={'ds': 'timestamp', 'Web Response': 'value'}, inplace=True)
result['Модель'] = 'Prophet'
result['Метрика'] = 'Web Response'
result['is_anomaly']=result['anomaly']
result['is_anomaly']=result['is_anomaly'].astype(bool)
# Сохраняем результат в CSV файл
result = result[['Модель', 'Метрика', 'timestamp', 'value',  'is_anomaly']]
result.to_csv('Prophet_Web Response.csv', index=False)

print("Результаты модели сохранены в файл 'result.csv'")

<ipython-input-23-fa1a17b7f87d>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-fa1a17b7f87d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-fa1a17b7f87d>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-fa1a17b7f87d>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Результаты модели сохранены в файл 'result.csv'


In [24]:
result

,Модель,Метрика,timestamp,value,is_anomaly
0,Prophet,Web Response,2024-04-15 23:32:00,0.007541,False
1,Prophet,Web Response,2024-04-15 23:33:00,0.008417,False
2,Prophet,Web Response,2024-04-15 23:34:00,0.009546,False
3,Prophet,Web Response,2024-04-15 23:35:00,0.009014,False
4,Prophet,Web Response,2024-04-15 23:36:00,0.012279,False
...,...,...,...,...,...
43267,Prophet,Web Response,2024-05-16 00:54:00,0.007921,False
43268,Prophet,Web Response,2024-05-16 00:55:00,0.009280,False
43269,Prophet,Web Response,2024-05-16 00:56:00,0.010534,False
43270,Prophet,Web Response,2024-05-16 00:57:00,0.009379,False


In [21]:
result.shape

(43272, 5)